In [1]:
import os 
import subprocess
import pandas as pd 

In [11]:
def retrive_ref(tax,taxid=False):
    if taxid == False:
        out = subprocess.Popen(f"esearch -db assembly -query '{tax}[ORGN]' | esummary | xtract -pattern DocumentSummary \
                -def NA -element Organism,AssemblyAccession,Taxid,assembly-status -block Stat \
                -if Stat@category -equals contig_count -or Stat@category -equals contig_l50 \
                -or Stat@category -equals contig_n50 -or Stat@category -equals total_length \
                -element Stat", shell=True, stdout=subprocess.PIPE).communicate()[0].decode('utf-8')
    else:
        tax=int(tax)
        out = subprocess.Popen(f"esearch -db assembly -query 'taxid{tax}' | esummary | xtract -pattern DocumentSummary \
                -def NA -element Organism,AssemblyAccession,Taxid,assembly-status -block Stat \
                -if Stat@category -equals contig_count -or Stat@category -equals contig_l50 \
                -or Stat@category -equals contig_n50 -or Stat@category -equals total_length \
                -element Stat", shell=True, stdout=subprocess.PIPE).communicate()[0].decode('utf-8')
    print(out)
    raw_lst = []
    lst = out.strip().split("\n")
    for i in lst:
        raw_lst.append(i.split("\t"))
    df = pd.DataFrame(raw_lst)
    df.columns =['Organism', 'AssemblyAccession', 
                    'Taxid', 'assembly-status', 'contig_count',
                    'contig_l50', 'contig_n50', 'total_length']

    df["Organism"].unique()
    df = df.astype({"contig_count": int})
    return df

def filter_taxa_df(df):
    print("Please Select a Assembly level: ")
    counter = 1
    uniques = df["assembly-status"].unique()
    for i in uniques:
        print(f"{counter}: {i}")
        counter += 1
    n = int(input("\n"))
    m = n-1

    out = df[df["assembly-status"] == df["assembly-status"].unique()[m]].sort_values("contig_count")
    
    print("Please Select a Number for arganism: ")
    counter = 1
    uniques = out["Organism"].unique()
    for i in uniques:
        print(f"{counter}: {i}")
        counter += 1
    n = int(input("\n"))
    m = n-1
    out2 = out[out["Organism"] == out["Organism"].unique()[m]].sort_values("contig_count")
    return out2

def download_ref(df):
    df2 = df[["AssemblyAccession", "total_length", "assembly-status", "contig_count", "contig_l50", "contig_n50"]]
    df2.index = df2.reset_index(drop = True).index + 1
    print("Please Select a Number to Download ref: ")
    print(df2)
    n = int(input("\n"))
    acc = test2.iloc[n]["AssemblyAccession"]
    subprocess.Popen(f"esearch -db assembly -query {acc} | elink -target nucleotide -name assembly_nuccore_refseq | efetch -format fasta > {acc}.fa", shell=True,stdout=subprocess.PIPE)
    print("Done")

In [12]:
# df = retrive_ref("Agrobacterium tumefaciens")
retrive_ref(358,taxid=True)

ValueError: Length mismatch: Expected axis has 1 elements, new values have 8 elements

In [4]:
test = filter_taxa_df(df)

Please Select a Assembly level: 
1: Contig
2: Chromosome
3: Complete Genome
4: Scaffold
Please Select a Number for arganism: 
1: Agrobacterium tumefaciens (a-proteobacteria)
2: Agrobacterium radiobacter K84 (a-proteobacteria)


In [5]:

test

,Organism,AssemblyAccession,Taxid,assembly-status,contig_count,contig_l50,contig_n50,total_length
15,Agrobacterium tumefaciens (a-proteobacteria),GCF_017726655.1,358,Complete Genome,2,1,2913790,5082709
18,Agrobacterium tumefaciens (a-proteobacteria),GCF_014489975.1,358,Complete Genome,2,1,2922353,5109662
205,Agrobacterium tumefaciens (a-proteobacteria),GCF_001551895.1,358,Complete Genome,2,1,2983661,5481595
5,Agrobacterium tumefaciens (a-proteobacteria),GCA_017897985.1,358,Complete Genome,3,1,2773415,5529348
168,Agrobacterium tumefaciens (a-proteobacteria),GCF_005221425.1,358,Complete Genome,3,1,2953857,5526017
177,Agrobacterium tumefaciens (a-proteobacteria),GCF_003667905.1,358,Complete Genome,3,1,3027766,5446032
176,Agrobacterium tumefaciens (a-proteobacteria),GCF_003667925.1,358,Complete Genome,4,2,2096799,5939134
178,Agrobacterium tumefaciens (a-proteobacteria),GCF_003666465.1,358,Complete Genome,4,2,2095493,5870997
170,Agrobacterium tumefaciens (a-proteobacteria),GCF_005221385.1,358,Complete Genome,4,1,3053646,5515917
179,Agrobacterium tumefaciens (a-proteobacteria),GCF_003666445.1,358,Complete Genome,4,1,2867449,5683743


In [ ]:
test2 = test[["AssemblyAccession", "total_length", "assembly-status", "contig_count", "contig_l50", "contig_n50", "FtpPath"]]
test2.index = test2.reset_index(drop = True).index + 1
test2


In [ ]:

acc = test2.iloc[0]["AssemblyAccession"]
acc
# subprocess.Popen(f"esearch -db assembly -query {acc} | elink -target nucleotide -name assembly_nuccore_refseq | efetch -format fasta > {acc}.fa", shell=True,stdout=subprocess.PIPE)

In [8]:
name = "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/286/275/GCA_000286275.1_ASM28627v1"
name = name + "/" + name.split("/")[-1] + "_genomic.fna.gz/"

In [9]:
name

'ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/286/275/GCA_000286275.1_ASM28627v1/GCA_000286275.1_ASM28627v1_genomic.fna.gz/'

In [ ]:
genomes/all/GCA/000/286/275/GCA_000286275.1_ASM28627v1/GCA_000286275.1_ASM28627v1_genomic.fna.gz